# Debugging autoreload

In [ ]:
%load_ext autoreload
%autoreload 2

# Load packages

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import plotly.express as px
import statsmodels.formula.api as smf
import plotly.graph_objects as go
from scripts.python.routines.manifest import get_manifest
from scripts.python.routines.plot.save import save_figure
from scripts.python.routines.plot.layout import add_layout
from statsmodels.stats.multitest import multipletests
import plotly.io as pio
pio.kaleido.scope.mathjax = None
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=False)
from scipy.stats import mannwhitneyu, median_test, kruskal, wilcoxon, friedmanchisquare
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.patheffects as path_effects
import random
import pathlib
from tqdm import tqdm
from src.utils.plot.bioinfokit import mhat, volcano
import gseapy as gp
import mygene
from sklearn.decomposition import PCA, IncrementalPCA, KernelPCA, TruncatedSVD
from sklearn.decomposition import MiniBatchDictionaryLearning, FastICA
from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection
from sklearn.manifold import MDS, Isomap, TSNE, LocallyLinearEmbedding
import upsetplot
from matplotlib_venn import venn2, venn2_circles
from itertools import chain
from sklearn.metrics import mean_absolute_error
from scripts.python.routines.plot.colorscales import get_continuous_color
from impyute.imputation.cs import fast_knn
import plotly
from scripts.python.routines.plot.p_value import add_p_value_annotation
from scripts.python.routines.sections import get_sections
from statannotations.Annotator import Annotator
import functools
import matplotlib.lines as mlines
import patchworklib as pw


def conjunction(conditions):
    return functools.reduce(np.logical_and, conditions)


def disjunction(conditions):
    return functools.reduce(np.logical_or, conditions)

# Load data

## Immunology

In [ ]:
path = f"D:/YandexDisk/Work/pydnameth/datasets/GPL21145/GSEUNN"
path_save = f"{path}/special/060_EpiSImAge"
pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)

df = pd.read_excel(f"{path_save}/df.xlsx", index_col=0)

feats_imm = pd.read_excel(f"{path}/data/immuno/feats_con.xlsx", index_col=0).index.values
feats_imm_fimmu = pd.read_excel(f"{path}/data/immuno/models/SImAge/feats_con_top10.xlsx", index_col=0).index.values
feats_imm_slctd = pd.read_excel(f"{path}/special/059_imm_data_selection/feats_selected.xlsx", index_col=0).index.values

## Epigenetics

In [ ]:
feats_epi = {}

### Train dataset

#### Correct index and save to binary

In [ ]:
df_epi = pd.read_csv(f"{path_save}/GSEUNN/betas.csv", index_col=0).transpose()
df_epi = df_epi.loc[df['Sample_Name'].values, :]
df_epi['Index'] = df.index.values
df_epi.set_index('Index', inplace=True)
df_epi.to_pickle(f"{path_save}/GSEUNN/betas.pkl")

#### Load corrected from binary

In [ ]:
df_epi = pd.read_pickle(f"{path_save}/GSEUNN/betas.pkl")
feats_epi['GSEUNN'] = df_epi.columns.values

### Test datasets

#### GSE87571

##### Correct index and save to binary

In [ ]:
df_gse_epi = pd.read_csv(f"{path_save}/GSE87571/betas.csv", index_col=0).transpose()
df_gse_phn = pd.read_csv(f"{path_save}/GSE87571/pheno.csv", index_col=0)
if not df_gse_epi.index.equals(df_gse_phn.index):
    print("Indexes are not equal!")
else:
    df_gse_epi.to_pickle(f"{path_save}/GSE87571/betas.pkl")
    df_gse_phn.to_pickle(f"{path_save}/GSE87571/pheno.xlsx")

##### Load corrected from binary

In [ ]:
df_gse_epi = pd.read_pickle(f"{path_save}/GSE87571/betas.pkl")
feats_epi['GSE87571'] = df_gse_epi.columns.values

#### GSE40279

##### Correct index and save to binary

In [ ]:
df_gse_epi = pd.read_csv(f"{path_save}/GSE40279/betas.csv", index_col=0).transpose()
df_gse_phn = pd.read_csv(f"{path_save}/GSE40279/pheno.csv", index_col=0)
if not df_gse_epi.index.equals(df_gse_phn.index):
    print("Indexes are not equal!")
df_gse_phn['Sample_Name'] = df_gse_phn.index.values
df_gse_phn.set_index('gsm', inplace=True)
df_gse_epi = df_gse_epi.loc[df_gse_phn['Sample_Name'].values, :]
df_gse_epi['Index'] = df_gse_phn.index.values
df_gse_epi.set_index('Index', inplace=True)

In [ ]:
df_gse_epi.to_pickle(f"{path_save}/GSE40279/betas.pkl")
df_gse_phn.to_pickle(f"{path_save}/GSE40279/pheno.xlsx")

##### Load corrected from binary

In [ ]:
df_gse_epi = pd.read_pickle(f"{path_save}/GSE40279/betas.pkl")
feats_epi['GSE40279'] = df_gse_epi.columns.values